# Working with ISIC Images
Images are at the heart of the ISIC Archive, and this notebook
will give you a sense of what the ```isicarchive``` package
can do in terms of retrieving images from the archive, and the
kinds of methods you can apply to them.
## Preparation
We'll start again by creating an ```IsicApi``` object, which
allows us to access the various pieces of information and images
stored in the archive. The call to ```cache_images()``` will
then download information (and meta information) about all
images that are available--if you are logged in as a regular
user, you may have access to additional images! **If you're
using the ```cache_images()``` method, please use a different
cache folder if you want to work with both publicly available
and private images in separate sessions.**

Next, we'll select all images from the ```IsicApi``` object that
have a specific diagnosis, and then display those images.

In [ ]:
# Settings (please adapt to your needs!)
username = None
password = None
# cache_folder = 'C:\\Users\\weberj3\\Documents\\ISIC'
cache_folder = '/Data/MSKCC/isicarchive/cache'

# Preparation
from isicarchive import IsicApi, func
api = IsicApi(username, password, cache_folder=cache_folder)

In [ ]:
# caching images and segmentations; this can take a long time!!
# so you may want to skip this cell for now...
api.cache_images()
api.cache_segmentations()

In [ ]:
# list studies
api.studies

In [ ]:
# load a study and cache the images associated with it
study = api.study('ISIC Annotation Study - All Features')
study.cache_imagedata()

In [ ]:
# load image and data
image = api.image(study.images[0])
image.load_imagedata()
image.load_superpixels()
image.map_superpixels()

In [ ]:
# create superpixel outlines as SVG paths
superpixel_colors = func.superpixel_colors()
outline_svg = func.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvg',
    path_attribs=superpixel_colors)
outlines = func.superpixel_outlines(
    image.superpixels['map'],
    image.superpixels['idx'].shape,
    out_format='osvgp',
    path_attribs=superpixel_colors)

# display one path
outlines[0]

In [ ]:
# write into an SVG file
with open('/tmp/test.svg', 'w') as file:
    file.write(outline_svg)

In [ ]:
# demostrate how to color a single superpixel
image_data = image.data
image_shape = image_data.shape
image_data.shape = (image_shape[0] * image_shape[1], -1)
map = image.superpixels['map']
superpixel_index = 472
pixel_count = map[superpixel_index, -1]
superpixel_pixels = map[superpixel_index, 0:pixel_count]
image_data[superpixel_pixels, 0] = 255
image_data[superpixel_pixels, 1] = 0
image_data[superpixel_pixels, 2] = 0
image_data.shape = image_shape

import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(image_data)
plt.show()